# Exploratory Analysis

In [1]:
import numpy as np
import pandas as pd
%load_ext autoreload
%autoreload 2
import os

In [2]:
from olist.data import Olist
data = Olist().get_data()

Each transaction on the Olist ecommerce platform is characterized by:
- a `customer_id`, that would buy...
- various`product_id`...
- to a `seller_id`...
- and leaves a `review_id`...
- all this belonging to an `order_id`

## 1 - Run an automated exploratory analysis with [ydata-profiling](https://github.com/ydataai/ydata-profiling)

In [5]:
# First, let's install the ydata-profiling package
! pip install --quiet pandas==1.4.4 ydata_profiling==4.2.0 Pillow==9.2.0

ERROR: Could not find a version that satisfies the requirement ydata_profiling==4.2.0 (from versions: 4.7.0, 4.8.3, 4.9.0, 4.10.0, 4.11.0, 4.12.0, 4.12.1, 4.12.2, 4.13.0, 4.14.0, 4.15.0, 4.15.1, 4.16.0, 4.16.1)
ERROR: No matching distribution found for ydata_profiling==4.2.0


In [6]:
# Then create a "reports" directory
!mkdir reports

mkdir: cannot create directory ‘reports’: File exists


In [7]:
# let's import ProfileReport from ydata_profiling
from ydata_profiling import ProfileReport
# choose which datasets to profile
datasets_to_profile = ['orders', 'products', 'sellers',
                  'customers', 'order_reviews',
                  'order_items']

👉 Create and save one `html report` per dataset to profile 

⏳ (It usually takes a few minutes)

In [8]:
for d in datasets_to_profile:
    print('exporting: '+ d)
    profile = ProfileReport(data[d], title = d)
    profile.to_file(f"reports/{d}_report.html")

exporting: orders


Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

...

exporting: order_items


Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

## 2 - Investigate the cardinalities of your various DataFrames

❓ **How many unique `orders`, `reviews`, `sellers`, `products` and `customer` is there ?**  
(You can use ydata-profiling or pandas methods on your notebook if you prefer)

In [14]:
for d in datasets_to_profile:
    if d == 'orders':
        print(f"Unique Orders: {data[d]['order_id'].nunique()}")
    elif d == 'order_reviews':
        print(f"Unique Reviews: {data[d]['review_id'].nunique()}")
    elif d == 'sellers':
        print(f"Unique Sellers: {data[d]['seller_id'].nunique()}")
    elif d == 'products':
        print(f"Unique Products: {data[d]['product_id'].nunique()}")
    elif d == 'customers':
        print(f"Unique Customers: {data[d]['customer_id'].nunique()}")

Unique Orders: 99441
Unique Products: 32951
Unique Sellers: 3095
Unique Customers: 99441
Unique Reviews: 98410


❓ **How many reviews is there per order? Do we have reviews for all orders ?**
<details>
    <summary markdown='span'>Hints</summary>

This info is not directly accessible in your individual csv. You'll need to proceed to merge
</details>

In [ ]:

# Merge the orders and order_reviews datasets on 'order_id', keeping all orders
merged_data = pd.merge(data['orders'], data['order_reviews'], on='order_id', how='left')

# Count reviews per order (this counts how many reviews each order has)
reviews_per_order = merged_data.groupby('order_id').size()

# Count orders with missing reviews (i.e., where review_id is NaN)
orders_with_reviews = merged_data[merged_data['review_id'].notna()]

# Number of orders with reviews
num_orders_with_reviews = orders_with_reviews['order_id'].nunique()

# Number of orders without reviews (where review_id is NaN)
num_orders_without_reviews = merged_data['order_id'].nunique() - num_orders_with_reviews

# Print the total number of reviews per order
print(f"Total number of reviews per order:\n{reviews_per_order}")

# Print whether every order has a review
if num_orders_with_reviews == len(data['orders']):
    print("Every order has a review.")
else:
    print(f"Some orders are missing reviews. {num_orders_without_reviews} orders have no reviews.")


Total number of reviews per order:
order_id
00010242fe8c5a6d1ba2dd792cb16214    1
00018f77f2f0320c557190d7a144bdd3    1
000229ec398224ef6ca0657da4fc703e    1
00024acbcdf0a6daa1e931b038114c75    1
00042b26cf59d7ce69dfabb4e55b4fd9    1
                                   ..
fffc94f6ce00a00581880bf54a75a037    1
fffcd46ef2263f404302a634eb57f7eb    1
fffce4705a9662cd70adb13d4a31832d    1
fffe18544ffabc95dfada21779c9644f    1
fffe41c64501cc87c801fd61db3f6244    1
Length: 99441, dtype: int64
Some orders are missing reviews. 768 orders have no reviews.


🧪 **Test your code below**

Store the number of orders with missing reviews as `int` in a variable named `n_missing_reviews`

In [19]:
n_missing_reviews = int(num_orders_without_reviews)

In [20]:
from nbresult import ChallengeResult

result = ChallengeResult('exploratory',
    n=n_missing_reviews
)
result.write()
print(result.check())


============================= test session starts ==============================
platform linux -- Python 3.12.9, pytest-8.3.4, pluggy-1.5.0 -- /home/saranjthilak92/.pyenv/versions/3.12.9/envs/lewagon/bin/python
cachedir: .pytest_cache
rootdir: /home/saranjthilak92/code/saranjthilak/04-Decision-Science/01-Project-Setup/data-exploratory-analysis/tests
plugins: typeguard-4.4.2, anyio-4.8.0
collecting ... collected 1 item

test_exploratory.py::TestExploratory::test_n_missing_reviews PASSED      [100%]

============================== 1 passed in 0.02s ===============================


💯 You can commit your code:

git add tests/exploratory.pickle

git commit -m 'Completed exploratory step'

git push origin master

